# Neurons example via low-level, flexible interface
## Prepare

In [1]:
from bayes_window import models
from bayes_window.fitting import fit_numpyro
from bayes_window.generative_models import generate_fake_spikes
import numpy as np
from sklearn.preprocessing import LabelEncoder

trans = LabelEncoder().fit_transform

## Make some data


In [2]:
df, df_monster, index_cols, firing_rates = generate_fake_spikes(n_trials=2,
                                                                n_neurons=8,
                                                                n_mice=4,
                                                                dur=7, )


df['log_isi'] = np.log10(df['isi'])

from bayes_window import visualization, utils
from importlib import reload

reload(visualization)
reload(utils)
y = 'isi'
df['neuron'] = df['neuron'].astype(int)
ddf, dy = utils.make_fold_change(df,
                                 y=y,
                                 index_cols=('stim', 'mouse_code', 'neuron'),
                                 treatment_name='stim',
                                 do_take_mean=True)

visualization.plot_data(x='neuron', y=dy, color='mouse_code', add_box=True, df=ddf)

alt.LayerChart(...)

## Estimate model

In [3]:
# y = list(set(df.columns) - set(index_cols))[0]
trace = fit_numpyro(y=df[y].values,
                    treatment=(df['stim']).astype(int).values,
                    condition=trans(df['neuron']),
                    group=trans(df['mouse']),
                    progress_bar=True,
                    model=models.model_hierarchical,
                    n_draws=100, num_chains=1, )

sample: 100%|██████████| 1100/1100 [00:05<00:00, 187.03it/s, 31 steps of size 2.07e-01. acc. prob=0.89]


## Add data back

In [4]:
reload(utils)
df_both, trace = utils.add_data_to_posterior(df,
                                             posterior=trace.posterior,
                                             y=y,
                                             fold_change_index_cols=['neuron', 'stim', 'mouse_code', ],
                                             treatment_name='stim',
                                             b_name='b_stim_per_condition',
                                             posterior_index_name='neuron',
                                             add_data=True
                                             )

## Plot data and posterior

In [6]:
# BayesWindow.plot_posteriors_slopes(df_both, y=f'{y} diff', x='neuron',color='mouse_code',title=y,hold_for_facet=False,add_box=False)


chart_d = visualization.plot_data(df=df_both, x='neuron', y=f'{y} diff', color='mouse_code')
chart_d

alt.LayerChart(...)

In [7]:
chart_p = visualization.plot_posterior(df=df_both, title=f'd_{y}', x='neuron', )
chart_p

alt.LayerChart(...)

In [8]:
(chart_d + chart_p).resolve_scale(y='independent')

alt.LayerChart(...)

In [9]:
(chart_d + chart_p).facet(column='neuron')

alt.FacetChart(...)